In [1]:
from IPython.core.debugger import set_trace
import numpy as np
import sys
import collections
import re
from pprint import pprint
f=open('day23test2.txt') #not with read because thats probably the whole file
lines = [line.rstrip('\n') for line in f]

In [2]:
import functools
from types import FunctionType

def copy_func(f):
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f,FunctionType): return copy(f)
    fn = FunctionType(f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__)
    fn.__dict__.update(f.__dict__)
    return fn

def patch_to(cls, as_prop=False):
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple,list)): cls=(cls,)
    def _inner(f):
        for c_ in cls:
            nf = copy_func(f)
            # `functools.update_wrapper` when passing patched function to `Pipeline`, so we do it manually
            for o in functools.WRAPPER_ASSIGNMENTS: setattr(nf, o, getattr(f,o))
            nf.__qualname__ = f"{c_.__name__}.{f.__name__}"
            setattr(c_, f.__name__, property(nf) if as_prop else nf)
        return f
    return _inner

def patch(f):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls)(f)

In [3]:
bots = []
power = []
for line in lines:
    result =[]
    result.append(int(re.search('pos\=\<(.*)',line.split(',')[0])[1]))
    result.append(int(line.split(',')[1]))
    result.append(int(line.split(',')[2][0:-1]))
    power.append(int(re.search('r\=(.*)',line.split(',')[3])[1]))
    bots.append(result)


In [4]:
from dataclasses import dataclass
from urllib.parse import SplitResult
splits = 0
@dataclass
class Block():
    left:int
    right:int
    up:int
    leftrange:int
    rightrange:int
    uprange:int
    
    lazy:int
    val:int
    subblocks:list
    level:int
    def iscontainedby(self, other):
        return (other.left <= self.left and other.left+other.leftrange >= self.left + self.leftrange and
            other.right <= self.right and other.right + other.rightrange >= self.right + self.rightrange and
            other.up <= self.up and other.up + other.uprange >= self.up + self.uprange)
    
    def excludes(self, other):
        return (other.left > self.left + self.leftrange or other.left + other.leftrange < self.left or
                other.right > self.right + self.rightrange or other.right + other.rightrange < self.right or
                other.up > self.up + self.uprange or other.up + other.uprange < self.up)
        
    def split(self):
        global splits
        splits += 1
        midl = self.left + self.leftrange //2 + 1
        midr = self.right + self.rightrange //2 + 1
        midu = self.up + self.uprange // 2 + 1
        for l in (self.left, midl+1):
            for r in (self.right, midr+1):
                for u in (self.up, midu+1):
                    self.subblocks.append(type(self)(l,r,u,self.leftrange//2, self.rightrange//2, self.uprange//2,0,0,[], self.level+1))
        # print(f'level {self.level} exploded')
        
    def __repr__(self):
        return self.level
    
    def add(self, block):
        if not self.leftrange:
            return
        # print(self.leftrange)
        if self.iscontainedby(block):
            # print('contains')
            self.val += 1
            self.lazy += 1
        elif not self.excludes(block):
            # print('subby')
            if not self.subblocks:
                self.split()
            for b in self.subblocks:
                b.add(block)
    
    def size(self):
        maxx = self.lazy
        if self.subblocks:
            maxx += max(b.size() for b in self.subblocks)
        return maxx
        
    def brightest(self):
        if self.leftrange == 1 and self.rightrange == 1 and self.uprange == 1:
            return self.val
        else:
            if not self.subblocks:
                return self.lazy
            else:
                return self.lazy + max(b.brightest() for b in self.subblocks)
    


In [5]:
@patch
def hi(self:Block):
    print('hi')
    
b= Block(0,0,0,0,0,0,0,0,[],0)

In [6]:
from re import L, U


@dataclass
class Robot():
    l:int
    r:int
    u:int
    p:int
    
    def reaches(self,l,r,u):
        return (abs(self.l-l) + abs(self.r-r) + abs(self.u - u)) <= self.p
        

class RoboBlock(Block):
    def iscontainedby(self, other):
        for l in (self.left, self.left+self.leftrange):
            for r in (self.right, self.right+ self.rightrange):
                for u in (self.up, self.up+self.uprange):
                    if not other.reaches(l,r,u):
                        return False
        return True
    
    def excludes(self, other):
        for l in (self.left, self.left+self.leftrange):
            for r in (self.right, self.right+ self.rightrange):
                for u in (self.up, self.up+self.uprange):
                    if other.reaches(l,r,u):
                        return False
        
        return not (self.left <= other.l <= self.left + self.leftrange and 
                self.right <= other.r <= self.right + self.rightrange and
                self.up <= other.u <= self.up + self.uprange)
        

In [7]:
1+1

2

In [8]:
from aocutils import grid
maxx = 0
robots = []
for (x,y,z), p in zip((bots), power):
    maxx = max(maxx, max(x,y,z)+p)
    print(maxx,x,y,z,p)
    robots.append(Robot(x,y,z,p))

14 10 12 12 2
16 12 14 12 2
20 16 12 12 4
20 14 14 14 6
250 50 50 50 200
250 10 10 10 5


In [9]:
1+1

2

In [10]:
size = 2
level = 0
while size < maxx:
    size *= 2
    
    level += 1
size, level

(256, 7)

In [11]:
b = RoboBlock(0,0,0,size,size,size,0,0,[],1)
for r in robots:
    b.add(r)

In [12]:
b.level

1

In [13]:
1+1

2